In [2]:
import os
from pathlib import Path
from PIL import Image
import torch
from torchvision.transforms import ToTensor

In [5]:
import matplotlib.pyplot as plt
import torchvision.utils as utils

def crop_and_save(image, index, p, dest):
    c, h, w = image.shape
    hn = h // p
    wn = w // p
    for i in range(hn): # 0...30
        for j in range(wn): # 0...32
            if c == 4:
            	tile = image[1:, (i*p):(i*p)+p, (j*p):(j*p)+p]
            else:
                tile = image[:, (i*p):(i*p)+p, (j*p):(j*p)+p]
            utils.save_image(tile, os.path.join(dest, f"{(index)*(hn*wn)+(i*wn)+j}.png"))


p = 224
root = "C:\\Users\\web\\Documents\\gaofen\\Test"

folders = ["Annotation__index", "Annotation__color", "Image__8bit_NirRGB"]
dest = "Patches_224x224"
for folder in folders:
    Path(os.path.join(root, dest, folder)).mkdir(parents = True, exist_ok=True)
for folder in folders:
    source = os.path.join(root, folder)
    image_files = [item for item in os.listdir(source)]
    for i,image in enumerate(image_files):
        image_path = os.path.join(source, image)
        im = ToTensor()(Image.open(image_path))
        crop_and_save(im, i, p,os.path.join(root, dest, folder))


In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
class Cropped5BP(Dataset):
	def __init__(self, root):
		self.image_dir = os.path.join(root, 'Image__8bit_NirRGB')
		self.index_dir = os.path.join(root, 'Annotation__index')
		self.color_dir = os.path.join(root, 'Annotation__color')
	def __len__(self):
		files = [os.path.join(self.image_dir,item) for item in os.listdir(path=self.image_dir)]		
		return len(files)
	def __getitem__(self, idx):
		image = ToTensor()(Image.open(os.path.join(self.image_dir, str(idx)+".png")))
		index = ToTensor()(Image.open(os.path.join(self.index_dir, str(idx)+".png")))
		color = ToTensor()(Image.open(os.path.join(self.color_dir, str(idx)+".png")))
		return image, index, color
dataset = Cropped5BP("C:\\Users\\web\\Documents\\gaofen\\Test\\Patches_224x224")
train_loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=1)
for image, index, color in train_loader:
	plt.imshow(image.squeeze().permute(1,2,0))	
	plt.show()
	plt.imshow(color.squeeze().permute(1,2,0))
	plt.show()
